In [1]:
import psycopg2 as pg2
import pandas as pd
import glob
from tqdm import tqdm
tqdm.pandas()
import gc

# con = pg2.connect(host='aminer_postgres_db',
#                   user='postgres',
#                   password='password',
#                   database='postgres')
# con.autocommit = True
# cur = con.cursor()

# def select(sql):
#     return pd.read_sql(sql,con)

Check running processes

In [2]:
# sql = '''
# SELECT pid , query, *
# FROM pg_stat_activity pgsa
# WHERE pid <> pg_backend_pid()
# '''

In [3]:
# df = select(sql)

### Create tables

In [2]:
class Database():
    def __init__(self,df_tmp, name_table):

        self.df_ = df_tmp
        self.name_table = name_table
        print(self.name_table)
        columns=[col for col in self.df_.columns if col not in ['references']]
        self.df_ = self.df_[columns]
        columns = {col:'TEXT' for col in self.df_.columns}
        columns.update({'id':'SERIAL PRIMARY KEY'})
        self.columns = columns
        self.con = pg2.connect(host='aminer_postgres_db',
                  user='postgres',
                  password='password',
                  database='postgres')
        self.con.autocommit = True
        self.cur = self.con.cursor()
    
    # запросы в бд
    def request_db(self, sql):
        return pd.read_sql(sql,self.con)
    
    # создание таблицы
    def create_paper(self):
        sub_q = ', '.join(' '.join((k, self.columns[k])) for k,v in self.columns.items())
        q = f'''CREATE TABLE IF NOT EXISTS {self.name_table} ({sub_q});'''
        print(q)
        self.cur.execute(q)
        self.delete_all_rows()
    
    # удаление таблицы
    def empty(self):
        q=f"DROP table IF EXISTS {self.name_table};"
        self.request_db(q)

    # удаление 1ой строчки
    def delete_row(self,id_):
        q=f"DELETE FROM {self.name_table} WHERE id=='{id_}'"
        self.request_db(q)

    # удаление нескольких строк
    def delete_many_rows(self,id_list):
        id_tuple = tuple(id_ for id_ in id_list)
        q=f"DELETE FROM {self.name_table} WHERE id in {id_tuple}"
        self.request_db(q)
        
    # удаление нескольких строк без удаления самой таблицы
    def delete_all_rows(self):
        q=f"DELETE FROM {self.name_table}"
        self.cur.execute(q)
    
    # сохрание в бд
    def record_data(self, dfsm):
        for index, dic in tqdm(dfsm.iterrows()):
            values = []
            for col in dfsm.columns:
                if self.columns[col] == 'TEXT':
                    values.append(str(dic[col]))
                else: 
                    values.append(int(float(dic[col])))
            data_tuple = list(values)
            ques = data_tuple
            for i, qu in enumerate(ques):
                if qu == None or qu == 'nan':
                    ques[i] = ''
                else:
                    ques[i] = ques[i].replace('\'', '')
            sub_q = ', '.join(col for col in list(dfsm.columns))
            q=f'''INSERT INTO {self.name_table}({sub_q}) VALUES {tuple(ques)};'''
            self.cur.execute(q)
        
#             if index % 100 == 0:
#                 print(f'{len(dfsm) - index} rows left')
#                 pass
        return True

    def insertion(self,dfsm):
        try:
            return self.record_data(dfsm)
        except Exception as error:
            print(f'{error=}')
            return False

    def insert_all(self):
        err_list = []
        
        dfsm = self.df_
        
        rt = self.insertion(dfsm)
        return err_list
   
    # метод для проверки бд
    def select_head(self):
        q = f'select * from {self.name_table} limit 10'
        return pd.read_sql(q, self.con)

In [4]:
# чтение файлов

In [5]:
for file in glob.glob('../data/data_csv/*'):
    df_tmp = pd.read_csv(file)
    # ограничил 1000 строчек
    df_tmp = df_tmp[:1000]
    ###################################
    idx_file = 'Paper_' + file.split('/')[-1].split('.')[0].split('_')[-1]
    db = Database(df_tmp, idx_file)
    db.create_paper()
    err_list = db.insert_all()
    del df_tmp
    gc.collect()
    

Paper_00
CREATE TABLE IF NOT EXISTS Paper_00 (_id TEXT, title TEXT, venue TEXT, year TEXT, keywords TEXT, n_citation TEXT, lang TEXT, authors TEXT, fos TEXT, page_start TEXT, page_end TEXT, volume TEXT, issue TEXT, issn TEXT, isbn TEXT, doi TEXT, pdf TEXT, url TEXT, abstract TEXT, id SERIAL PRIMARY KEY);


1000it [00:03, 288.24it/s]


Paper_01
CREATE TABLE IF NOT EXISTS Paper_01 (_id TEXT, title TEXT, authors TEXT, venue TEXT, year TEXT, keywords TEXT, fos TEXT, n_citation TEXT, page_start TEXT, page_end TEXT, lang TEXT, volume TEXT, issue TEXT, issn TEXT, isbn TEXT, doi TEXT, pdf TEXT, url TEXT, abstract TEXT, id SERIAL PRIMARY KEY);


1000it [00:05, 172.57it/s]


Paper_02
CREATE TABLE IF NOT EXISTS Paper_02 (_id TEXT, title TEXT, authors TEXT, venue TEXT, year TEXT, keywords TEXT, fos TEXT, n_citation TEXT, page_start TEXT, page_end TEXT, lang TEXT, volume TEXT, issue TEXT, issn TEXT, isbn TEXT, doi TEXT, pdf TEXT, url TEXT, abstract TEXT, id SERIAL PRIMARY KEY);


...

1000it [00:05, 173.36it/s]
/opt/conda/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (4,10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Paper_09
CREATE TABLE IF NOT EXISTS Paper_09 (_id TEXT, title TEXT, authors TEXT, venue TEXT, year TEXT, keywords TEXT, fos TEXT, n_citation TEXT, page_start TEXT, page_end TEXT, lang TEXT, volume TEXT, issue TEXT, issn TEXT, isbn TEXT, doi TEXT, url TEXT, abstract TEXT, pdf TEXT, id SERIAL PRIMARY KEY);


1000it [00:06, 156.95it/s]
/opt/conda/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (18) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Paper_10
CREATE TABLE IF NOT EXISTS Paper_10 (_id TEXT, title TEXT, authors TEXT, venue TEXT, year TEXT, keywords TEXT, fos TEXT, page_start TEXT, page_end TEXT, volume TEXT, issn TEXT, doi TEXT, url TEXT, abstract TEXT, pdf TEXT, issue TEXT, n_citation TEXT, lang TEXT, isbn TEXT, id SERIAL PRIMARY KEY);


1000it [00:05, 183.82it/s]


In [6]:
db.select_head()

,_id,title,authors,venue,year,keywords,fos,page_start,page_end,volume,issn,doi,url,abstract,pdf,issue,n_citation,lang,isbn,id
0,5e8d90649fced0a24b5e2476,Wavefront Control of 2D Curved Coding Metasurf...,"[{_id: 54092051dabfae450f46388f, name: Yang Zh...","{sid: IEEE ACCESS, issn: 2169-3536, t: J, raw:...",2019.0,"[Curved metasurfaces, wavefront control, array...","[Aperture, Wavefront, Lattice (order), Effecti...",158427.0,158433.0,7,2169-3536,10.1109/ACCESS.2019.2905656,[db/journals/access/access7.html#ZhouXCZYLZZD1...,We extend the array theory to design 2D curved...,,,,,,1
1,5e8d90649fced0a24b5e24dd,The Study on Internal Flow Characteristics of ...,"[{name: Gao Zhenjun, sid: Gao Zhenjun}, {name:...","{sid: journals/access, t: J, raw: IEEE Access}",2019.0,,"[Impeller, Flow (psychology), Turbulence, Magn...",100003,100013,7,,10.1109/ACCESS.2019.2928307,"[db/journals/access/access7.html#ZhenjunCJH19,...",,,,,,,2
2,5e8d90649fced0a24b5e250e,SoK of Used Cryptography in Blockchain.,"[{name: Mayank Raikwar, org: Norwegian Univ Sc...","{sid: IEEE ACCESS, issn: 2169-3536, t: J, raw:...",2019.0,"[Blockchain, cryptography, hash function, proo...","[Computer security, Computer science, Cryptogr...",148550.0,148575.0,7,2169-3536,10.1109/ACCESS.2019.2946983,"[db/journals/access/access7.html#RaikwarGK19, ...",The underlying fundaments of blockchain are cr...,https://static.aminer.cn/storage/pdf/arxiv/19/...,,,,,3
3,5e8d90649fced0a24b5e24cd,Iterative Multiuser Detection and Decoding for...,"[{name: Dongbo Li, org: Harbin Inst Technol, S...","{sid: IEEE ACCESS, issn: 2169-3536, t: J, raw:...",2020.0,"[SCMA, SEFDM, IoT, iterative multiuser detecti...","[Computer science, Frequency-division multiple...",24887,24895,8,2169-3536,10.1109/ACCESS.2020.2964041,"[db/journals/access/access8.html#LiJWGG20, htt...",Sparse code multiple access (SCMA) combined wi...,,,,,,4
4,5e8d90649fced0a24b5e251e,PB-NILM: Pinball Guided Deep Non-Intrusive Loa...,"[{name: Eduardo Gomes, org: M ITI, ITI, LARSyS...","{sid: IEEE ACCESS, issn: 2169-3536, t: J, raw:...",2020.0,"[Neural networks, Benchmark testing, Home appl...","[Data mining, Search engine, Convolution, Comp...",48386.0,48398.0,8,2169-3536,10.1109/ACCESS.2020.2978513,"[db/journals/access/access8.html#GomesP20, htt...",The work in this paper proposes the applicatio...,,,,,,5
5,5e8d90649fced0a24b5e2536,Improvement of the Prediction-Based Energy Eff...,"[{name: Numukobwa Segolene, sid: Numukobwa Seg...","{sid: journals/access, t: J, raw: IEEE Access}",2019.0,,"[Data modeling, Microsoft Windows, Data transm...",156420,156429,7,,10.1109/ACCESS.2019.2948840,"[db/journals/access/access7.html#SegoleneLJ19,...",,,,,,,6
6,5e8d90659fced0a24b5e2588,A Naive-Bayes-Based Fault Diagnosis Approach f...,"[{_id: 53f633a4dabfaec2591b3afd, name: Wei He,...","{sid: IEEE ACCESS, issn: 2169-3536, t: J, raw:...",2020.0,"[Analog circuit, fault diagnosis, cross-wavele...","[Kernel (linear algebra), Analogue electronics...",5065.0,5079.0,8,2169-3536,10.1109/ACCESS.2018.2888950,"[db/journals/access/access8.html#HeHLZ20, http...",Analog circuit is one of the most commonly use...,,,,,,7
7,5e8d90659fced0a24b5e25c0,On Fault-Tolerant Control Systems: A Novel Rec...,"[{name: Jorge Rodriguez-Guerra, sid: Jorge Rod...","{sid: journals/access, t: J, raw: IEEE Access}",2020.0,,"[Control theory, Adaptive system, Fault detect...",39322,39335,8,,10.1109/ACCESS.2020.2975543,[db/journals/access/access8.html#Rodriguez-Gue...,,,,,,,8
8,5e8d90649fced0a24b5e2423,Liver Detection Algorithm Based on an Improved...,"[{name: Kaijian Xia, sid: Kaijian Xia, orcid: ...","{sid: journals/access, t: J, raw: IEEE Access}",2019.0,,"[Pattern recognition, Computer science, Segmen...",175135,175142,7,,10.1109/ACCESS.2019.2953517,"[db/journals/access/access7.html#XiaY19a, http...",,,,,,,9
9,5e8d90649fced0a24b5e243e,Low-Complexity High-Resolution Parameter Estim...,"[{name: Yu-Chien Lin, org: Natl Chiao Tung Uni...","{sid: IEEE ACCESS, issn: 2169-3536, t: J, raw:...",2020.

Kill all

In [5]:
sql = '''
SELECT 
    pg_terminate_backend(pid) 
FROM 
    pg_stat_activity 
WHERE 
    -- don't kill my own connection!
    pid <> pg_backend_pid()

'''

In [6]:
cur.execute(sql)